# Planning to TCP Poses 📐

In the previous notebook we saw how to plan paths between joint configurations.
In this notebook we will explore how planning to TCP poses opens doors to many interesting possibilities, such as:
- 💥 **Filtering** e.g. pregrasp configurations to ensure moving the the grasp pose does not collide with the environment
- 🥇 **Selecting** optimal paths, e.g. the shortest, smoothest, fastest after time parametrization or most clearance
- 📊 **Ranking** goal joint configurations and planning to them iteratively, selecting the first path that is found. This ranking could be based on the distance of the goal joint configuration to the a desirable joint configuration, e.g the start joint configuration.

**The 4-Step Process**

Let's break down how we approach TCP pose planning:
1. 🧮 **IK Solutions:** Calculate candidate goal joint configurations from the goal TCP pose.
2. 🤔 **Rank & Prioritize (Optional):** Rank those candidates
3. 🗺️ **Plan Paths:** Try planning paths to the candidates.
4. 🏆 **Select the Best (Optional):** Choose the optimal path based on your criteria.



<!-- **Why Planning to TCP Poses Matters**

- ✨ **Flexibility:** A single TCP pose often corresponds to multiple possible joint configurations for a 6-DOF robot.
- 💡 **Strategic choices:** This opens doors! We can filter pre-grasp configurations for collision-free grasps, optimize paths based on smoothness or speed, and more.


However, in this notebook we will show how planning to TCP poses opens interesting opportunities.

A key fact to know, is that for a given TCP pose in the workspace of a 6-DOF robot, there will be several joint configurations that can achieve that pose. 
For some robots, such as the universal robots we use at our lab, you can analytically calculate the inverse kinematics to find all the joint configurations that can achieve a given TCP pose.

This finite set of possible goal configurations provide an interesting opportunity.
For example, if the TCP pose we plan to is a pregrasp pose from which we want to move linearly to a grasp pose, we can filter the pregrasp joint configurations to only select the from which we can move to the grasp pose without colliding with the environment.

Another possiblity with multiple joint configurations is that we can plan paths to each of them and then select the best path according to some criteria. For example, we might want to select the shortest path, or the smoothest path, or the path that can be executed the fastest after time parametrization.

However, if any path is good enough, and we care mostly about planning time, we can rank the goal joint configurations, and plan to them iteratively, and select the first path that is found. This ranking could for example be based on the distance goal joint configuration to the a desirable joint configuration .e.g the start joint configuration.


The way we achieve planning to TCP poses is by using a four-step process:
1. Calculate the inverse kinematics of the goal TCP pose to find the candidate goal joint configurations.
2. Optional: rank the goal joint configurations
3. Attempt to plan a path to the goal joint configuration(s)
4. Optional: select the best path -->

## 1. Setting up the scene & collision checker 🏗️️

For more details about this, see the [`airo-drake`](https://github.com/airo-ugent/airo-drake) notebooks.


In [ ]:
import numpy as np
from pydrake.planning import RobotDiagramBuilder
from airo_drake import SingleArmScene, add_floor, add_manipulator, add_meshcat, finish_build
from pydrake.planning import SceneGraphCollisionChecker

robot_diagram_builder = RobotDiagramBuilder()

meshcat = add_meshcat(robot_diagram_builder)
arm_index, gripper_index = add_manipulator(robot_diagram_builder, "ur3e", "robotiq_2f_85", static_gripper=True)
add_floor(robot_diagram_builder)
robot_diagram, context = finish_build(robot_diagram_builder)

scene = SingleArmScene(robot_diagram, arm_index, gripper_index, meshcat)


collision_checker = SceneGraphCollisionChecker(
    model=scene.robot_diagram,
    robot_model_instances=[scene.arm_index, scene.gripper_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)

collision_checker.CheckConfigCollisionFree(np.zeros(6))

In [ ]:
start_joints = np.deg2rad([0, -90, 90, -90, -90, 0])

print(collision_checker.CheckConfigCollisionFree(start_joints))

plant = scene.robot_diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)
plant.SetPositions(plant_context, scene.arm_index, start_joints)
scene.robot_diagram.ForcedPublish(context)

## 2. Creating the motion planner 🧭

To enable planning to TCP poses, we need to provide an additional argument to the `SingleArmOmplPlanner` constructor, namely an `inverse_kinematics_fn` that returns a list of joint configurations for a given TCP pose.

### 2.1 `inverse_kinematics_fn` 🧮
In this notebook we will be using the analytical inverse kinematics from the [`ur-analytic-ik`](https://github.com/Victorlouisdg/ur-analytic-ik) Python package.

In [ ]:
from ur_analytic_ik import ur3e
from airo_typing import HomogeneousMatrixType, JointConfigurationType
from airo_drake import visualize_frame

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175  # 175 mm in z


def inverse_kinematics_fn(tcp_pose: HomogeneousMatrixType) -> list[JointConfigurationType]:
    solutions = ur3e.inverse_kinematics_with_tcp(tcp_pose, tcp_transform)
    return solutions


goal_pose = np.identity(4)

X = np.array([-1.0, 0.0, 0.0])
Y = np.array([0.0, 1.0, 0.0])
Z = np.array([0.0, 0.0, -1.0])
top_down_orientation = np.column_stack([X, Y, Z])
goal_pose[:3, :3] = top_down_orientation
goal_pose[:3, 3] = [0.17, 0.0, 0.0]

visualize_frame(scene.meshcat, "goal_pose", goal_pose)

solutions = inverse_kinematics_fn(goal_pose)

with np.printoptions(precision=3, suppress=True):
    for solution in solutions:
        print(solution)

In [ ]:
from airo_drake import animate_joint_configurations

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, solutions)

### 2.2 `SingleArmOmplPlanner` 🧭

In [ ]:
from airo_planner import SingleArmOmplPlanner


planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree, inverse_kinematics_fn=inverse_kinematics_fn)

## 3. Planning to TCP poses 🗺

### 3.1 Choosing the shortest path 📏

The default behavior of `SingleArmOmplPlanner`  is planning to all valid goal joint configurations and selecting the shortest path.

In [ ]:
path = planner.plan_to_tcp_pose(start_joints, goal_pose)

In [ ]:
animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, path)

**Double checking the path lengths:**

In [ ]:
from airo_drake import calculate_joint_path_length

path_lengths = [calculate_joint_path_length(p) for p in planner._all_paths]

print("Path lengths")
for i, path_length in enumerate(path_lengths):
    print(f"{i}: {path_length:.2f}")

print(f"\nLength of path that planner returned: {calculate_joint_path_length(path):.2f}")

### 3.2 Ranking goal configuration desirability 🔀

It's common to prefer that the robot arm stays close specific desirable configurations, such as:
- the start joint configuration 🏁
- a home configuration 🏡

We let the planner know these goal configuration preferences by setting the:
- `rank_goal_configurations_fn` attribute of the `SingleArmOmplPlanner`.

In `airo-planner` we provide a `rank_by_distance_to_desirable_configurations()` function that we can use for this purpose.

In [ ]:
from airo_planner.selection.goal_selection import rank_by_distance_to_desirable_configurations

ranked_solutions = rank_by_distance_to_desirable_configurations(solutions, [start_joints])

np.printoptions(precision=3, suppress=True)

print("Distances to ranked solutions")
for i, solution in enumerate(ranked_solutions):
    distance = np.linalg.norm(solution - start_joints)
    print(f"{distance:.2f}")

In [ ]:
planner.rank_goal_configurations_fn = rank_by_distance_to_desirable_configurations

path = planner.plan_to_tcp_pose(start_joints, goal_pose)

In [ ]:
print(f"\nLength of path that planner returned: {calculate_joint_path_length(path):.2f}")

In [ ]:
animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, path)

## 3.3 Filtering goal configurations 🧹

**Filtering** is possibliy the most powerful feature of planning to TCP poses.
By filtering the goal joint configurations, we can ensure that the robot goes to configurations that enable downstream tasks. 
For example, you want to make sure that end configuration of the path to the pregrasp pose is such that you can move linearly to the grasp pose without colliding with the environment.

In [ ]:
from airo_planner.interfaces import SingleArmPlanner
from pydrake.trajectories import Trajectory


from airo_drake import visualize_frame


# grasp_location = np.array([0.2, 0.5, 0.5]) # This moves the robot through a singularlity for some start configurations
grasp_location = np.array([0.2, 0.3, 0.2])

gripper_forward_direction = np.array([1, 0, 0])

Z = gripper_forward_direction / np.linalg.norm(gripper_forward_direction)
Y = np.array([0, 0, -1])  # 0, 0, 1 is also an option
X = np.cross(Y, Z)

grasp_orientation = np.column_stack([X, Y, Z])
grasp_pose = np.identity(4)
grasp_pose[0:3, 0:3] = grasp_orientation
grasp_pose[0:3, 3] = grasp_location

pregrasp_pose = grasp_pose.copy()
pregrasp_pose[0:3, 3] -= 0.25 * gripper_forward_direction

visualize_frame(scene.meshcat, "pregrasp_pose", pregrasp_pose)
visualize_frame(scene.meshcat, "grasp_pose", grasp_pose)

In [ ]:
from ur_analytic_ik import ur3e

tcp_transform = np.identity(4)
tcp_transform[2, 3] = 0.175  # 175 mm in z

start_configurations = ur3e.inverse_kinematics_with_tcp(pregrasp_pose, tcp_transform)
goal_configurations = ur3e.inverse_kinematics_with_tcp(grasp_pose, tcp_transform)

start_configurations = np.array(start_configurations).squeeze()
goal_configurations = np.array(goal_configurations).squeeze()

len(start_configurations), len(goal_configurations)

In [ ]:
def path_collisions_as_emojis(is_collision_free: list[bool]):
    """Displays an emoji-based visualization of a path's collisions.

    Example output: "✅✅💥✅✅✅💥✅✅✅✅"

    Args:
        is_collision_free: A list of booleans, where True indicates no collision.

    Returns:
        A string of emojis representing the collision status of the path.
    """
    emojis = ["✅" if is_free else "💥" for is_free in is_collision_free]
    emoji_str = "".join(emojis)
    return emoji_str


print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(start_configurations)))
print(path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(goal_configurations)))

In [ ]:
from pydrake.trajectories import PiecewisePose
from pydrake.math import RigidTransform
from airo_drake import discretize_drake_pose_trajectory


rigid_transforms = [RigidTransform(pose) for pose in [pregrasp_pose, grasp_pose]]
times = np.linspace(0, 1, len(rigid_transforms))
pose_trajectory = PiecewisePose.MakeLinear(times=times, poses=rigid_transforms)


tcp_path = discretize_drake_pose_trajectory(pose_trajectory).poses

for i, tcp_pose in enumerate(tcp_path):
    visualize_frame(scene.meshcat, f"tcp_path/pose_{i}", tcp_pose, length=0.05, opacity=0.1)

In [ ]:
from airo_drake import calculate_valid_joint_paths, calculate_joint_paths


def ur3e_inverse_kinematics(tcp_pose: HomogeneousMatrixType) -> list[JointConfigurationType]:
    solutions_1x6 = ur3e.inverse_kinematics_with_tcp(tcp_pose, tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions


joint_paths_ = calculate_joint_paths(tcp_path, ur3e_inverse_kinematics)

for i, path in enumerate(joint_paths_):
    print(f"{path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(path))}")

In [ ]:
joint_paths = calculate_valid_joint_paths(
    tcp_path, ur3e_inverse_kinematics, collision_checker.CheckConfigCollisionFree
)

for i, path in enumerate(joint_paths):
    print(f"{path_collisions_as_emojis(collision_checker.CheckConfigsCollisionFree(path))}")

In [ ]:
animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, joint_paths[0])

In [ ]:
from functools import partial

from airo_planner.selection.goal_selection import filter_with_distance_to_configurations

path_starts = [path[0] for path in joint_paths]

filter_fn = partial(filter_with_distance_to_configurations, joint_configurations_close=path_starts)

filter_fn(start_configurations)

In [ ]:
# use this a filtering example: we want to filter out the grasp paths that collide with the environment

# def plan_to_grasp(planner: SingleArmPlanner, grasp_pose: HomogeneousMatrixType) -> Trajectory:
#     pass
#     # attempt planning the the grasp pose directly
#     #   if all goals in collision try backing-off to a pregrasp pose
#     #   time parametrize both paths and concatenate them
from airo_typing import InverseKinematicsFunctionType, JointConfigurationCheckerType


def plan_pregrasp(
    grasp_pose: HomogeneousMatrixType,
    start_configuration: JointConfigurationType,
    inverse_kinematics_fn: InverseKinematicsFunctionType,
    is_state_valid_fn_pregrasp: JointConfigurationCheckerType,
    is_state_valid_fn_grasp: JointConfigurationCheckerType,
):

    pregrasp_distances_to_try = [0.05, 0.1, 0.15, 0.2, 0.25]

    planner = SingleArmOmplPlanner(is_state_valid_fn_pregrasp, inverse_kinematics_fn=inverse_kinematics_fn)

    for distance in pregrasp_distances_to_try:
        # 1. Compute pregrasp pose
        pregrasp_pose = grasp_pose.copy()
        pregrasp_pose[0:3, 3] -= distance * pregrasp_pose[0:3, 2]

        # 2. Compute grasp TCP path
        rigid_transforms = [RigidTransform(pose) for pose in [pregrasp_pose, grasp_pose]]
        times = np.linspace(0, 1, len(rigid_transforms))
        pose_trajectory = PiecewisePose.MakeLinear(times=times, poses=rigid_transforms)
        tcp_path = discretize_drake_pose_trajectory(pose_trajectory).poses

        # 3 Compute valid grasp joint paths
        joint_paths = calculate_valid_joint_paths(tcp_path, ur3e_inverse_kinematics, is_state_valid_fn_grasp)
        path_starts = [path[0] for path in joint_paths]

        # 4 plan to pregrasp tcp poses, filtering on the valid grasp joint paths
        filter_fn = partial(filter_with_distance_to_configurations, joint_configurations_close=path_starts)

        planner.filter_goal_configurations_fn = filter_fn
        path = planner.plan_to_tcp_pose(start_configuration, pregrasp_pose)

        if path is not None:
            return path

    # Exhausted all pregrasp poses to try


path = plan_pregrasp(
    grasp_pose,
    start_joints,
    ur3e_inverse_kinematics,
    collision_checker.CheckConfigCollisionFree,
    collision_checker.CheckConfigCollisionFree,
)

animate_joint_configurations(scene.meshcat, scene.robot_diagram, scene.arm_index, path)